In [2]:
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns

import time

In [3]:
# pandas display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

In [6]:
# Load the pickled cleaned encounters-level data
df = pd.read_pickle('../data_local/job_posts_clean.pkl')

In [7]:
df.head()

,company_descr,company_name,date_posted,email_content,email_date,email_from,email_subject,job_post_link,job_posting,job_title,location
0,At Bluecore we are transforming the way eComme...,Bluecore,3 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=6c7ebde0dbd8...,We are looking for Senior Data Scientists with...,Senior Data Scientist,"New York, NY 10002"
1,,Loadsmart,3 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=59a797491eb2...,"Who we are:\nAt Loadsmart, we won't settle for...",Data Scientist,"New York, NY"
2,,Foursquare,4 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=7490c513ac10...,About Foursquare:\nSince our inception in 2009...,Data Scientist,"New York, NY"
3,About HRG\n\nHRG's TRADING TEAM recruits the t...,"The Hagan-Ricci Group, Inc.",5 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=4eceac18fce0...,A small and very successful quantitative fund ...,Data Scientist / Signal Researcher,"New York, NY"
4,,Oath Inc,5 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=6b82f29c6741...,"Oath, a subsidiary of Verizon, is a values-led...",Data Scientist Intern,"New York, NY"


In [8]:
df.describe()

,company_descr,company_name,date_posted,email_content,email_date,email_from,email_subject,job_post_link,job_posting,job_title,location
count,183,183,183,183,183,183,183,183,183,183,183
unique,89,150,22,10,10,3,8,183,157,105,106
top,,Facebook,1 day ago,Indeed Job Alert data science jobs Jobs 1-30 ...,"Wed, 21 Feb 2018 18:14:45 -0600 (CST)",Indeed,30+ new data science jobs,https://www.indeed.com/viewjob?jk=fed71f97f1d7...,Intent Media isn’t your usual company. Our wor...,Data Scientist,"New York, NY"
freq,43,5,38,33,33,96,96,1,3,48,30


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 11 columns):
company_descr    183 non-null object
company_name     183 non-null object
date_posted      183 non-null object
email_content    183 non-null object
email_date       183 non-null object
email_from       183 non-null object
email_subject    183 non-null object
job_post_link    183 non-null object
job_posting      183 non-null object
job_title        183 non-null object
location         183 non-null object
dtypes: object(11)
memory usage: 15.8+ KB


In [10]:
df.company_name.value_counts()

Facebook                                 5
Loadsmart                                3
Intent Media                             3
Evoke Technologies                       2
The Hagan-Ricci Group, Inc.              2
S.C. International                       2
SES Satellites                           2
Axiologic Solutions                      2
KPMG                                     2
Ingram Micro                             2
CGI                                      2
Johnson & Johnson Family of Companies    2
SSC                                      2
Ingersoll Rand                           2
Pacific Northwest National Laboratory    2
Two Sigma                                2
OSRAM SYLVANIA                           2
Roku Inc.                                2
Two Sigma Investments, LLC.              2
L.A. Care Health Plan                    2
McKinsey & Company                       2
Capgemini: RO                            2
ExoTerra Resource                        2
Booz Allen 

In [11]:
df.job_title.value_counts()
# Standardize this 
# Maybe derive role and level from this

Data Scientist                                                             48
Data Science Intern                                                         5
Senior Data Scientist                                                       5
Data Scientist, Analytics (Instagram)                                       3
Analyst, Data Science                                                       3
Researcher, Deep Learning                                                   2
VP Data Science                                                             2
Data Scientist, SMB                                                         2
Post Doctorate RA - Statistician/Data Scientist                             2
Director, Data Scientist                                                    2
Digital Insights Data Scientist                                             2
Data Scientist - Engineering Modeling & Simulation                          2
Data Scientist/Governance                                       

In [12]:
df.location.value_counts()
# Derive country, state andd city into separate fields from this

New York, NY                  30
New York City, NY, US         10
Boston, MA                     7
San Francisco, CA              6
Irvine, CA                     4
Raleigh, NC                    4
Atlanta, GA                    4
Chicago, IL                    3
Richland, WA                   2
Beverly, MA                    2
Remote                         2
Morrisville, NC 27560          2
Los Angeles, CA                2
Springfield, VA                2
Bangalore, IN                  2
Greater New York City Area     2
Minnesota                      2
Los Gatos, CA, US              2
Plano, TX                      2
Princeton, NJ, US              2
Los Angeles, CA 90017          2
Washington, DC                 2
Silver Springs, MD             2
Menlo Park, CA                 2
Reston, VA                     2
                              ..
Buffalo, NY, US                1
McLean, VA                     1
Arlington, VA 22209            1
Boyers, PA 16018               1
Bedford, M

In [13]:
df.date_posted.value_counts()
# Calculate the actual dates from this - to be run on the day of scraping (02/22/18 for this run)

1 day ago              38
2 days ago             34
3 days ago             24
5 days ago             17
Posted 2 days ago      11
Posted 3 days ago      10
Posted 1 day ago        7
7 days ago              7
9 days ago              5
4 days ago              4
10 days ago             4
30+ days ago            4
Posted 21 hours ago     4
6 days ago              3
14 days ago             2
8 days ago              2
11 days ago             2
16 hours ago            1
Posted 20 hours ago     1
12 days ago             1
15 days ago             1
6 hours ago             1
Name: date_posted, dtype: int64

In [14]:
df.email_date.value_counts()
# not using this field for now, just keeping it for reference

Wed, 21 Feb 2018 18:14:45 -0600 (CST)    33
Mon, 19 Feb 2018 17:27:47 -0600 (CST)    33
Tue, 20 Feb 2018 19:23:41 -0600 (CST)    30
Mon, 19 Feb 2018 13:12:58 -0600 (CST)    23
Wed, 21 Feb 2018 13:40:20 -0600 (CST)    12
Thu, 22 Feb 2018 03:07:24 +0000 (UTC)    11
Tue, 20 Feb 2018 01:01:02 +0000 (UTC)    11
Wed, 21 Feb 2018 02:12:09 +0000 (UTC)    11
Thu, 22 Feb 2018 13:36:51 -0600 (CST)    10
Tue, 20 Feb 2018 13:51:08 -0600 (CST)     9
Name: email_date, dtype: int64